In [1]:
from ExampleAnalyzer.main import ExampleAnalyzer
from framework.AnalyzerUtilities import *

In [2]:
ana = ExampleAnalyzer()

--- load_feature_config from .csv files ---
load inFeatures csv:  /home/zchen/Documents/Analysis/nanoblt/ExampleAnalyzer/featureConfig/inFeature.csv
load internalFeatures csv:  /home/zchen/Documents/Analysis/nanoblt/ExampleAnalyzer/featureConfig/internalFeature.csv
load outFeatures csv:  /home/zchen/Documents/Analysis/nanoblt/ExampleAnalyzer/featureConfig/outFeature.csv
Successfully load all .csv files
--- lgenerate_cuda_struct_declaration ---
generate cuda struct 'MaskEvents' 
generate cuda struct 'Events' 
generate cuda struct 'EventsInternal' 
generate cuda struct 'EventsOut' 
Successfully generate all cuda struct 
--- compile_cuda_kernels in .cu files ---
Successfully complie cuda kernels


In [3]:
#infile = '/home/zchen/cms/data/EGamma_Run2018B/E4BC7705-C2F2-1D4D-BBD5-3E375685BA51.root'
infile = '../data/tt.root'
ana.process_infile(infile,'example.h5')

time mask 0.1738569736480713
pass mask 364876/855000, eff=0.427
time init_events read tree 2.8513753414154053
time init_events apply mask 0.19232559204101562
time init_events flat jaggedArray 0.1749858856201172
time init_events bool2int 0.0008294582366943359
time init_events cumsum 0.0016088485717773438
time init_events device soa 8.034706115722656e-05
time init_events internal-out 0.002501249313354492
this is event selection
time totalGPU 0.014807701110839844
this is post process
eventsOut is  {'nPassElectron': array([2, 1, 1, ..., 1, 1, 0], dtype=int32), 'nPassMuon': array([0, 2, 1, ..., 1, 0, 0], dtype=int32)}
this is store data example.h5
